In [1]:
class DNA():    
    def __init__(self, gene_size = 8):
        self.gene_size = gene_size
        
        def r(): return [np.random.randint(2, size=self.gene_size), None]
        
        self.genes = {
            'Size':r(),
            'Speed':r(), 
            'Max_stamina':r()
        }
        
        self.express_genes()
        
    def __str__(self):
        s = 'DNA:\n'
        for g in self.genes:
            s += g + ' ' +str(self.genes[g]) + '\n'
        return s
            
            
    def express_genes(self):
        mul = np.array([2**-i for i in range(self.gene_size)])
        for k in self.genes:
            self.genes[k][1] = DNA_MAX[k] * np.sum(np.multiply(self.genes[k][0], mul), axis=0)/sum(mul)
        return self
    
    def __add__(self, other):
        new = DNA(gene_size = self.gene_size)
        for g in new.genes:
            mask = np.random.randint(2, size=self.gene_size)
            new.genes[g][0] = np.multiply(self.genes[g][0], mask) + np.multiply(other.genes[g][0], 1- mask)
        new.express_genes()
        return new
    
    def get_genes(self):
        return self.genes['Size'][1], self.genes['Speed'][1], self.genes['Max_stamina'][1]
    
    def mutate(self, p=0.01):
        

class cinematics():
    def __init__(self, x, y, theta=0, v=0, max_v=0, omega=0):
        self.x = x
        self.y = y
        self.theta = theta # Angle
        self.v = v # Linear speed
        self.max_v = max_v # Max speed
        self.omega = omega # Rotation speed
        self.limits=None
        
    def __str__(self):
        return '(' + str(self.x) + ', ' + str(self.y) + ')'
        
    def update(self, d_v=0, omega=0, dt=1):
        self.v = max(0, min(self.v + d_v, self.max_v))
        self.omega = omega
        self.theta += self.omega * dt
        dx = self.v * dt * cos(radians(self.theta))
        dy = self.v * dt * sin(radians(self.theta))
        x = self.x
        y = self.y
        self.x, self.y = self.in_limits(x + dx, y + dy)
        return x, y, self.x, self.y
    
    def stop(self):
        self.v = 0
        self.omega = 0
    
    def arrow(self):
        return [self.x, self.y, radians(self.theta)]
    
    def set_limits(self, xmin=None, xmax=None, ymin=None, ymax=None, dim=None):
        if dim is not None:
            self.limits=[[0, dim[0]], [0, dim[1]]]
        else:
            self.limits = [[xmin, xmax], [ymin, ymax]]
        return self
    
    def in_limits(self, x, y):
        if self.limits is None:
            return x, y
        if x >= self.limits[0][1] or x <= self.limits[0][0]:
            self.theta += 90
        if y >= self.limits[1][1] or y <= self.limits[1][0]:
            self.theta += 90
        x = max(min(x, self.limits[0][1]), self.limits[0][0])
        y = max(min(y, self.limits[1][1]), self.limits[1][0])
        return x, y

In [2]:
class Animal():
    def __init__(self, pos, dna=None, age=0, dim=None, stamina=None):
        self.brain = Brain(BRAIN["In"], BRAIN["Out"])
        self.alive = True
        if dna is None:
            self.dna = DNA()
        else:
            self.dna = dna
        self.cin = cinematics(pos[0], pos[1], theta=pos[2], max_v=self.get_carac('Speed'))
        if dim is not None:
            self.cin.set_limits(dim=dim)
        self.decision = {
            'd_v': 0,
            'action': None
        }
        if stamina is None:
            self.stamina = self.get_carac('Max_stamina') / 3
        else:
            self.stamina = stamina
        self.sex = rd.randint(0, 1)
        self.age = age
        self.had_child = 0
        self.id = get_id()
        
    def __str__(self):
        return  str(self.id) + ' at ' + str(self.cin)
    
    def check_alive(self):
        if self.stamina <= 0:
            self.alive = False
            #print('Hungry', self.id)
        if self.age > DNA_MAX['Max_age']:
            #print('Old', self.id)
            self.alive = False
        return self
    
    def get_carac(self, key):
        if key in self.dna.genes:
            return self.dna.genes[key][1]
        
    def update_pos(self):
        self.check_alive()
        if self.alive:
            self.age +=1
            self.had_child = max(0, self.had_child - 1)
            self.update_decision()
            x, y, x_new, y_new = self.cin.update(d_v = self.decision['d_v'], omega = self.decision['omega'], dt=1)   
            d = dist((x, y), (x_new, y_new))
            self.stamina -= d / self.get_carac('Speed') * self.get_carac('Size')
            return x, y, x_new, y_new
        return self.cin.x, self.cin.y, self.cin.x, self.cin.y
        
    def update_decision(self, pattern = 'random'):
        self.check_alive()
        if self.alive:
            if pattern == 'random':
                self.decision = {
                    'd_v': (rd.random() *2 - 1) * 3,
                    'omega': (rd.random() *2 - 1) * 10,
                    'action': None
                }
        else:
            self.decision = {
                    'd_v': 0,
                    'omega': 0,
                    'action': None
                }
        return self
    
    def eat_grass(self, grass):
        self.stamina = min(self.get_carac('Max_stamina'), self.stamina + grass)
        return self
    
    def interact(self, other):
        self.check_alive()
        if self.alive == False:
            return 'Dead'
        if other.sex != self.sex:
            if self.age >= DNA_MAX["Mating_age"] and self.had_child == 0 and self.stamina >= self.get_carac('Max_stamina') / 3:
                return 'Mate'
            else:
                return 'Nothing'
        else:
            return 'Fight'
    
    def __add__(self, other):
        self.check_alive()
        if self.alive:
            dx, dy = (2* rd.random() - 1) * 10, (2* rd.random() - 1) * 10
            dim = [self.cin.limits[0][1], self.cin.limits[1][1]]
            stamina = self.get_carac('Max_stamina') / 3 + other.get_carac('Max_stamina') / 3
            self.stamina -= stamina/2
            other.stamina -= stamina/2
            new = Animal(dna = self.dna + other.dna, 
                         pos=[self.cin.x + dx, self.cin.y + dy, self.cin.theta], 
                         dim=dim, 
                         stamina=stamina)
            self.had_child, other.had_child = DNA_MAX["Mating_age"], DNA_MAX["Mating_age"]
            return new
    
    def __float__(self):
        return float(self.get_carac('Size'))